In [1]:
# Imports for data processing
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
from imblearn.pipeline import Pipeline

# Imports for evaluation metrics
from sklearn.metrics import roc_auc_score, accuracy_score

# Imports for ML models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.svm import SVC

# Imports for hyperparameter optimization
import optuna as opt

# Other imports
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle
import sys

# Ensure reproducibility
random_state = 0

/home/grnc13/miniconda3/envs/DataScience/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Import data and perform train and validation split

In [2]:
# Import data
datapath = "data/healthcare-dataset-stroke-data-train.csv"
df = pd.read_csv(datapath)
df

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,82.0,0,1,Yes,Private,Urban,144.90,26.4,smokes,1
1,Male,4.0,0,0,No,children,Rural,106.22,16.7,Unknown,0
2,Male,58.0,0,0,Yes,Private,Urban,79.95,25.9,never smoked,0
3,Female,20.0,0,0,No,Private,Rural,96.57,34.1,never smoked,0
4,Female,10.0,0,0,No,children,Rural,69.84,13.7,Unknown,0
...,...,...,...,...,...,...,...,...,...,...,...
4083,Female,51.0,0,0,Yes,Self-employed,Urban,232.89,34.0,smokes,0
4084,Male,64.0,0,1,Yes,Private,Urban,191.61,37.5,smokes,1
4085,Male,37.0,0,0,Yes,Self-employed,Rural,82.43,39.1,Unknown,0
4086,Female,22.0,0,0,No,Private,Rural,62.00,32.7,smokes,0


In [3]:
# Split the data into features and label
features = [col for col in list(df.columns) if col!="stroke"]
X, y = df[features], df["stroke"]

## Create the data preprocessor

Create the data preprocessing steps according to the EDA performed in file `1-eda.ipynb`:
- Impute missing values with the mean
- Perform one hot encoding on categorical columns

The EDA, however, misses one important step, which is the scaling of the data. It was not discussed in EDA because it actually makes it harder to understand the data. However, since we are using models that evaluate distances between datapoints, it is very important to scale our data.

Since the models we will be using do not assume normally distributed data, all features will be normalized in the [0, 1] range.

To summarize, the following steps are performed for preprocessing the data:
- Impute missing values with mean for the numerical columns
- One hot encoding for categorical columns
- MinMaxNormalization for every column

In [4]:
# Create the mean imputer and feature scaler
num_features = [col for col in features if df[col].dtype in ["int64", "float64"]]
imputer = SimpleImputer()
scaler = MinMaxScaler()

# Create numerical preprocessor
num_preprocessor = Pipeline(steps=[
    ("imputer", imputer),
    ("scaler", scaler)
])

# Create the one-hot encoder
one_hot_features = ["work_type", "smoking_status"]
one_hot_encoder = OneHotEncoder(handle_unknown="ignore")

# Create ordinal encoder
cat_features = [col for col in features if col not in num_features]
ordinal_features = [col for col in cat_features if col not in one_hot_features]
ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

# Create categorical preprocessor
cat_preprocessor = ColumnTransformer(transformers=[
    ("onehot", one_hot_encoder, one_hot_features),
    ("ordinal", ordinal_encoder, ordinal_features)
])

# Create the preprocessor
preprocessor = ColumnTransformer(transformers=[
    ("num_preprocessor", num_preprocessor, num_features),
    ("cat_preprocessor", cat_preprocessor, cat_features)
])

Finally, before going into the model selection phase, it is important to recall that the used dataset is imbalanced. More specifically, there are a lot fewer datapoints of patients who actually had a stroke. 

To compensate for their smaller frequency, we can increase their importance by giving them more weight. The following code cell calculates the class weights for the training data.

In [5]:
def get_class_weights(X, y):
    # Create the class weights due to data imbalance
    largest_class_size = max([X[y==c].shape[0] for c in y.unique()])
    class_weights = {c: largest_class_size/X[y==c].shape[0] for c in y.unique()}
    return class_weights
    
get_class_weights(X, y)

{1: 20.181347150259068, 0: 1.0}

However, some models are can't handle this type of argument to produce their results. For a more uniform approach across all model, an over-sampling technique, `SMOTE`, is applied using the `imblearn` package. This technique generates new datapoints for the minority class to balance the dataset.

In [6]:
# Create dataset balancer
resampler = SMOTE(random_state=random_state)

In [7]:
# Check that SMOTE balances the dataset
# NOTE: SMOTE here is applied to all data, later the data is split into training and validation, only training data should use SMOTE
X_res, y_res = resampler.fit_resample(preprocessor.fit(X, y).transform(X), y)
get_class_weights(X_res, y_res)

{1: 1.0, 0: 1.0}

As we can see from the class weights, SMOTE does indeed balance our dataset!

## Model selection

To perform model selection, we will use cross validation to better evaluate our models on validation data. One thing to note is that we can not use SMOTE in our validation pipeline, only for training. This is because we want to train the model to give the same importance to each class, but to evaluate it in a dataset as close to the test set as possible (imbalaced). To do this, we will write a costum cross validation function:

In [8]:
def cross_validate(X, y, pipe, n_splits, verbose=False):
    # Create folds for cross-validation
    kf = KFold(n_splits=n_splits)

    # Iterate folds for cross-validation
    train_score, val_score = 0, 0
    kfold_bar = enumerate(kf.split(X))
    if verbose:
        kfold_bar = tqdm(kfold_bar, total=n_splits, desc="Cross-validating", position=1, file=sys.stdout)
    for fold, (train_idx, val_idx) in kfold_bar:
        
        # Define training and validation sets
        X_train, X_val = X.loc[train_idx], X.loc[val_idx]
        y_train, y_val = y.loc[train_idx], y.loc[val_idx]
        
        # Fit pipeline to training data
        pipe = pipe.fit(X_train, y_train)
        
        # Get trained preprocessor and model
        preprocessor = pipe.get_params()["preprocessor"]
        model = pipe.get_params()["model"]
        
        # Save train data score
        y_train_pred = pipe.predict(X_train)
        train_score += roc_auc_score(y_train, y_train_pred) / n_splits
        
        # Define validation pipeline (remove SMOTE)
        val_pipe = Pipeline(steps=[
            ("preprocessor", preprocessor),
            ("model", model)
        ])
        
        # Save validation data score
        y_val_pred = val_pipe.predict(X_val)
        val_score += roc_auc_score(y_val, y_val_pred) / n_splits
        
    return train_score, val_score

To perform model selection, a few models were considered to solve this binary classification problem:
- Logistic regression classifier
- Support vector classifier
- Random forest classifier

They are all fitted to the training data according to the class weights and later evaluated on the validation dataset using the area under the ROC curve.

In [9]:
# Create model pool to select from
model_pool = [
    {"name": "LR", "model": LogisticRegression(random_state=random_state), "train_score": 0, "val_score": 0},
    {"name": "SVM", "model": SVC(random_state=random_state), "train_score": 0, "val_score": 0},
    {"name": "RFC", "model": RandomForestClassifier(random_state=random_state), "train_score": 0, "val_score": 0},
    {"name": "KNN", "model": KNeighborsClassifier(), "train_score": 0, "val_score": 0},
    {"name": "DTC", "model": DecisionTreeClassifier(random_state=random_state), "train_score": 0, "val_score": 0},
    {"name": "GNB", "model": GaussianNB(), "train_score": 0, "val_score": 0},
    {"name": "XGB", "model": XGBClassifier(), "train_score": 0, "val_score": 0},
    {"name": "NN", "model": MLPClassifier(random_state=random_state, max_iter=500, early_stopping=True), "train_score": 0, "val_score": 0}
]

# Define the metric function and number of kfold splits
metric = roc_auc_score
n_splits = 5

# Iterate each model in the pool
model_bar = tqdm(model_pool, total=len(model_pool), desc="Iterating models", position=0, file=sys.stdout)
for model_dict in model_bar:
    
    # Get dictionary values
    name, model = model_dict["name"], model_dict["model"]
    
    # Define the training pipeline using the model
    train_pipe = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("resampler", resampler),
        ("model", model)
    ])
    
    # Cross validate
    train_score, val_score = cross_validate(X, y, train_pipe, n_splits, True)
    model_dict["train_score"], model_dict["val_score"] = train_score, val_score
        
    # Print final results
    tqdm.write(f"Model: {name} | Train score: {train_score:.3f} | Val score: {val_score:.3f}")

Cross-validating: 100%|██████████| 5/5 [00:00<00:00,  6.89it/s]
Model: LR | Train score: 0.776 | Val score: 0.759      
Cross-validating: 100%|██████████| 5/5 [00:04<00:00,  1.11it/s]
Model: SVM | Train score: 0.829 | Val score: 0.678             
Cross-validating: 100%|██████████| 5/5 [00:02<00:00,  1.94it/s]
Model: RFC | Train score: 1.000 | Val score: 0.556             
Cross-validating: 100%|██████████| 5/5 [00:00<00:00,  7.09it/s]
Model: KNN | Train score: 0.937 | Val score: 0.588             
Cross-validating: 100%|██████████| 5/5 [00:00<00:00, 20.15it/s]
Model: DTC | Train score: 1.000 | Val score: 0.572             
Cross-validating: 100%|██████████| 5/5 [00:00<00:00, 39.87it/s]
Model: GNB | Train score: 0.649 | Val score: 0.647             
Cross-validating: 100%|██████████| 5/5 [00:01<00:00,  3.02it/s]
Model: XGB | Train score: 0.983 | Val score: 0.554             
Cross-validating: 100%|██████████| 5/5 [00:12<00:00,  2.56s/it]
Model: NN | Train score: 0.826 | Val score: 0.67

As we can see from these trained models, the one that faired better was the Logistic Regressor. However, both the Support Vector Machine classifier and the Neural Network also performed quite well. The Neural Network is especially promising, as it is very likely that performing hyperparameter tuning will boost its performance quite significantly.

## Hyperparameter optimization

Now that we know what the best models are, we will try to find their best hyperparameters so that we can get them to perform the best they can. We will be tuning:
- Logistic Regression Classifier
- Support Vector Machine Classifier
- Neural Networks Classifier

In order to do so, we will use an optimization package called `optuna`. First of all, let's define funtions to create the models and define their hyperparameter spaces:

In [10]:
def LR_model(trial):
    solver = trial.suggest_categorical("solve", ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]) # Solver hyperparameter
    c = trial.suggest_float("c", 1e-3, 1) # Hyperparameter for regularization strength
    model = LogisticRegression(solver=solver, C=c, random_state=random_state)
    return model
    
def SVM_model(trial):
    kernel = trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"]) # Define kernel
    c = trial.suggest_float("c", 1e-3, 1) # Hyperparameter for regularization strength
    model = SVC(C=c, kernel=kernel, probability=True, random_state=random_state)
    return model

def NN_model(trial):
    num_hidden_layers = trial.suggest_int("num_hidden_layers", 1, 4) # Define number of hidden layers
    hidden_layer_sizes = [trial.suggest_int(f"hidden_layer_{i}", 5, 20) for i in range(num_hidden_layers)] # Define features per hidden layer
    model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, random_state=random_state, max_iter=500, early_stopping=True)
    return model

Using these functions, we can now create an objective function, which we will use to optimize the ROC AUC metric for all of our models.

In [11]:
def objective(trial, name):
    # Select model based on its name
    if name == "LR":
        model = LR_model(trial)
    elif name == "SVM":
        model = SVM_model(trial)
    else:
        model = NN_model(trial)
        
    # Define training pipeline on the model
    train_pipe = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("resampler", resampler),
        ("model", model)
    ])
    
    # Perform cross validation
    n_splits = 5
    _, val_score = cross_validate(X, y, train_pipe, n_splits)
    
    return val_score

Now we will create our model pool and perform hyperparameter optimization for each model:

In [12]:
# Create model pool
model_pool = {
    "LR": None,
    "SVM": None,
    "NN": None
}

# Iterate each model in the pool
model_bar = tqdm(model_pool, total=len(model_pool), desc="Iterating models", position=0, file=sys.stdout)
for name in model_bar:
    
    # Set tqdm bar postfix to model name
    model_bar.set_postfix(model=name)
    
    # Create optuna study
    sampler = opt.samplers.TPESampler(seed=random_state)
    study = opt.create_study(study_name=name, direction="maximize", sampler=sampler)
    
    # Optimize
    foo = lambda trial: objective(trial, name)
    study.optimize(foo, n_trials=20)
    model_pool[name] = study

Iterating models:   0%|          | 0/3 [00:00<?, ?it/s, model=LR]

[I 2022-10-31 21:16:36,783] A new study created in memory with name: LR
[I 2022-10-31 21:16:37,342] Trial 0 finished with value: 0.7581401653833015 and parameters: {'solve': 'lbfgs', 'c': 0.6462482189535894}. Best is trial 0 with value: 0.7581401653833015.
[I 2022-10-31 21:16:37,785] Trial 1 finished with value: 0.7653168630553093 and parameters: {'solve': 'liblinear', 'c': 0.5293660248331515}. Best is trial 1 with value: 0.7653168630553093.
[I 2022-10-31 21:16:38,485] Trial 2 finished with value: 0.758656456868515 and parameters: {'solve': 'lbfgs', 'c': 0.83278722570239}. Best is trial 1 with value: 0.7653168630553093.
[I 2022-10-31 21:16:38,957] Trial 3 finished with value: 0.7576260189209498 and parameters: {'solve': 'liblinear', 'c': 0.780748647110169}. Best is trial 1 with value: 0.7653168630553093.
[I 2022-10-31 21:16:39,345] Trial 4 finished with value: 0.7605686766291944 and parameters: {'solve': 'sag', 'c': 0.4152472780505331}. Best is trial 1 with value: 0.7653168630553093.
[

Iterating models:  33%|███▎      | 1/3 [00:09<00:19,  9.89s/it, model=SVM]

[I 2022-10-31 21:16:46,672] A new study created in memory with name: SVM
[I 2022-10-31 21:16:59,392] Trial 0 finished with value: 0.6982987188640801 and parameters: {'kernel': 'poly', 'c': 0.4242311445395658}. Best is trial 0 with value: 0.6982987188640801.
[I 2022-10-31 21:17:33,676] Trial 1 finished with value: 0.5630916620467079 and parameters: {'kernel': 'sigmoid', 'c': 0.3840580773069519}. Best is trial 0 with value: 0.6982987188640801.
[I 2022-10-31 21:18:14,356] Trial 2 finished with value: 0.5817470400624714 and parameters: {'kernel': 'sigmoid', 'c': 0.07196502213968906}. Best is trial 0 with value: 0.6982987188640801.
[I 2022-10-31 21:18:30,695] Trial 3 finished with value: 0.688945537929285 and parameters: {'kernel': 'rbf', 'c': 0.8701421360985724}. Best is trial 0 with value: 0.6982987188640801.
[I 2022-10-31 21:18:43,613] Trial 4 finished with value: 0.7658166360725196 and parameters: {'kernel': 'linear', 'c': 0.11915615144306428}. Best is trial 4 with value: 0.765816636072

Iterating models:  67%|██████▋   | 2/3 [05:54<03:26, 206.70s/it, model=NN] 

[I 2022-10-31 21:22:31,142] A new study created in memory with name: NN
[I 2022-10-31 21:22:35,584] Trial 0 finished with value: 0.698303555454844 and parameters: {'num_hidden_layers': 3, 'hidden_layer_0': 16, 'hidden_layer_1': 14, 'hidden_layer_2': 13}. Best is trial 0 with value: 0.698303555454844.
[I 2022-10-31 21:22:38,231] Trial 1 finished with value: 0.7304941459762713 and parameters: {'num_hidden_layers': 2, 'hidden_layer_0': 15, 'hidden_layer_1': 12}. Best is trial 1 with value: 0.7304941459762713.
[I 2022-10-31 21:22:43,106] Trial 2 finished with value: 0.6225403157490155 and parameters: {'num_hidden_layers': 4, 'hidden_layer_0': 20, 'hidden_layer_1': 11, 'hidden_layer_2': 17, 'hidden_layer_3': 13}. Best is trial 1 with value: 0.7304941459762713.
[I 2022-10-31 21:22:47,577] Trial 3 finished with value: 0.6956147124140829 and parameters: {'num_hidden_layers': 3, 'hidden_layer_0': 19, 'hidden_layer_1': 6, 'hidden_layer_2': 6}. Best is trial 1 with value: 0.7304941459762713.
[I 2

Iterating models: 100%|██████████| 3/3 [07:01<00:00, 140.56s/it, model=NN]


As we can see from the hyperparameter optimization, the following were the best hyperparameters for each model:
- Logistic Regression: solver = liblinear and c = 0.3191852813529867
- Support Vector Machine: kernel = linear and c = 0.8549785503094043
- Neural Network: hidden_layer_sizes = [9, 19]

## Training the models with the best hyperparameters

Now that the best model hyperparameters have been tuned using cross-validation, we can discard the validation data to train these models in the entire training dataset (including the previous validation data):

In [17]:
# Create list of best models
best_models = {
    ("lr", LogisticRegression(solver="liblinear", C=0.3191852813529867, random_state=random_state)),
    ("svc", SVC(kernel="linear", C=0.8549785503094043, probability=True, random_state=random_state)),
    ("nn", MLPClassifier(hidden_layer_sizes=[9, 19], random_state=random_state, max_iter=500, early_stopping=True))
}

# Iterate each one of them
model_bar = tqdm(best_models, total=len(best_models), desc="Training best models", position=0, file=sys.stdout)
for name, model in model_bar:
    
    # Define training pipeline on the model
    train_pipe = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("resampler", resampler),
        ("model", model)
    ])
    
    # Fit the training pipeline on the ENTIRE training dataset (no validation)
    train_pipe = train_pipe.fit(X, y)
    
    # Create test pipe without the resampler
    trained_preprocessor = train_pipe.get_params()["preprocessor"]
    trained_model = train_pipe.get_params()["model"]
    test_pipe = Pipeline(steps=[
        ("preprocessor", trained_preprocessor),
        ("model", trained_model)
    ])
    
    # Save the trained test pipeline in the models folder
    with open(f"models/{name}.pkl", "wb") as f:
        pickle.dump(test_pipe, f, protocol=pickle.HIGHEST_PROTOCOL)

Training best models: 100%|██████████| 3/3 [00:04<00:00,  1.65s/it]
